In [1]:
import ee
import os
import json
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
load_dotenv()

ee.Authenticate()
projectName = os.getenv("PROJECT")
ee.Initialize(project=projectName)

In [6]:
i_date = '2022-01-01'
f_date = '2022-01-10'
scale = 1000
collection_name = 'MODIS/061/MOD11A1'

# Load MODIS LST dataset
lst = ee.ImageCollection(collection_name).select('LST_Day_1km').filterDate(i_date, f_date)
geometry = ee.Geometry.Polygon([
        [
            [
                120.74772644100017,
                17.15755081300017
            ],
            [
                120.68620300300017,
                17.166650772000025
            ],
            [
                120.66735077000021,
                17.193929673000127
            ],
            [
                120.68167114300002,
                17.198799133000136
            ],
            [
                120.68264770500002,
                17.255729676000158
            ],
            [
                120.61428833100001,
                17.310699462000173
            ],
            [
                120.55795288100012,
                17.307779313000083
            ],
            [
                120.53884124800004,
                17.357610703000034
            ],
            [
                120.57779693600003,
                17.42794990600015
            ],
            [
                120.58556366000016,
                17.476249695000092
            ],
            [
                120.52526092500023,
                17.49980926600017
            ],
            [
                120.47895050000011,
                17.481849670000088
            ],
            [
                120.46806335400015,
                17.514780045000123
            ],
            [
                120.48757171600005,
                17.545000076000008
            ],
            [
                120.5045928960003,
                17.64545059300002
            ],
            [
                120.58183288500015,
                17.799249650000036
            ],
            [
                120.69017028700011,
                17.834899903000064
            ],
            [
                120.72754669100004,
                17.892890931000125
            ],
            [
                120.78321838400018,
                17.911699295000062
            ],
            [
                120.8332366940001,
                17.959980012000074
            ],
            [
                120.9097366330002,
                17.950580597000112
            ],
            [
                120.92783355600011,
                17.978069305000133
            ],
            [
                120.98722839400023,
                17.939619064999988
            ],
            [
                121.0101699820002,
                17.843719482000154
            ],
            [
                121.06674957300004,
                17.848140716000046
            ],
            [
                121.09979248100012,
                17.824209212000028
            ],
            [
                121.12270355300006,
                17.683950424999978
            ],
            [
                121.09150695800008,
                17.666749954000124
            ],
            [
                121.11972045800019,
                17.58934974700003
            ],
            [
                121.11019134600008,
                17.56086921700006
            ],
            [
                120.99800872700018,
                17.49139022900016
            ],
            [
                120.99442291200023,
                17.40873909100003
            ],
            [
                120.89766693000013,
                17.190799714000036
            ],
            [
                120.79891204800015,
                17.187879562000035
            ],
            [
                120.74772644100017,
                17.15755081300017
            ]
        ]
    ])


In [7]:
def reduce_image(image, geometry):
    date = ee.Date(image.get('system:time_start')).format('YYYY-MM-dd')
    mean_lst = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=geometry,
        scale=scale,
        bestEffort=True
    ).get('LST_Day_1km')
    return ee.Feature(None, {'date': date, 'LST_Day_1km': mean_lst})


In [8]:
province="Abra"

In [10]:
time_series = lst.map(lambda img: reduce_image(img, geometry=geometry)).filter(ee.Filter.notNull(['LST_Day_1km']))
time_series

In [12]:
df = pd.DataFrame(time_series.getInfo()['features'])
df

,type,geometry,id,properties
0,Feature,None,2022_01_01,"{'LST_Day_1km': 15028.415017153844, 'date': '2..."
1,Feature,None,2022_01_02,"{'LST_Day_1km': 14914.083702810678, 'date': '2..."
2,Feature,None,2022_01_04,"{'LST_Day_1km': 14984.564663367895, 'date': '2..."
3,Feature,None,2022_01_07,"{'LST_Day_1km': 14917.312062413444, 'date': '2..."
4,Feature,None,2022_01_08,"{'LST_Day_1km': 14961.649549618824, 'date': '2..."
5,Feature,None,2022_01_09,"{'LST_Day_1km': 14920.23394195098, 'date': '20..."


In [13]:
df = pd.json_normalize(df['properties'])
df

,LST_Day_1km,date
0,15028.415017,2022-01-01
1,14914.083703,2022-01-02
2,14984.564663,2022-01-04
3,14917.312062,2022-01-07
4,14961.649550,2022-01-08
5,14920.233942,2022-01-09
